In [1]:
import json
import math
import pandas as pd

#from text_extraction import file_finder, TextExtraction
from data_creator import create_data
from utils import get_context, few_shot_cot, get_fewshot_cot_examples, get_label
from dotenv import load_dotenv
from pathlib import Path

from sklearn.metrics import f1_score
from tqdm import tqdm

In [2]:
df_dict = create_data(500, 30)

def filter_label(dataframes_dict: dict[pd.DataFrame], label: int) -> pd.DataFrame:
    # Create an empty list to store filtered DataFrames
    filtered_dataframes = []
    
    for key, df in dataframes_dict.items():
        if 'Label' in df.columns:
            filtered_df = df[df['Label'] == label]
            
            filtered_dataframes.append(filtered_df)
    
    result_df = pd.concat(filtered_dataframes, ignore_index=True)
    
    return result_df

def sample_data(dataframe):
    sampled_df = dataframe.sample(n=100, random_state=42) 
    return sampled_df



opinionated_data = filter_label(df_dict, 1)
neutral_data = sample_data(filter_label(df_dict, 0))

In [3]:
df = pd.concat([opinionated_data, neutral_data], ignore_index=True)

In [4]:
def get_precictions(df):
    predictions = []
    for i in tqdm(range(len(df))):
        #name = df["Authors"].iloc[i]
        title = df["Title"].iloc[i]
        context = get_context(df["context"].iloc[i])
        footnote = df["footnote_text"].iloc[i]
        #pred = context_sentiment(context)
        pred = few_shot_cot(examples = get_fewshot_cot_examples(), citation=context, title=title, footnote=footnote)
        
        #while pred != "0" and pred != "1":
            #print("Retrying prediction...")
            #pred = few_shot_cot(examples = get_fewshot_cot_examples(), citation=context, title=title, footnote=footnote)
        pred = get_label(pred)
        predictions.append(pred)
        #predictions = [int(i) for i in predictions]           
    return predictions

predictions = get_precictions(neutral_data)

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:03<05:41,  3.45s/it]

The label "neutral" is appropriate for this citation because the author does not make any statement about the quality or value of the cited work. The citation simply describes the action taken by Kenyon in sending a draft of AJC resolutions to Sir Louis Mallet in Paris. There is no expression of personal opinion or judgment about the work or its authors, making it a neutral citation.


  2%|▏         | 2/100 [00:06<04:55,  3.01s/it]

The label for this citation is "neutral." The author of the citation does not make any statement about the quality or value of the cited work by Léon Duguit. The citation provides a summary of historical events and sociological determinants without expressing any personal opinion or judgment about the work itself. The focus is on presenting information about the impact of military mobilization on political sociology during the Great War, rather than evaluating the quality of the cited work or its author. Therefore, the citation is neutral in nature.


...

The label "opinionated" is appropriate for this citation because the author makes a positive judgment about the impact of new community schemes on social leadership. Phrases like "helped to foster 'unexpected sources of social leadership'" indicate the author's evaluation of the outcome of these schemes, suggesting a positive view of the social changes brought about by the initiatives. This positive assessment of the social impact reflects the author's opinion, making the citation opinionated.


 99%|█████████▉| 99/100 [04:11<00:02,  2.57s/it]

The label for this citation is "neutral." The reason for this classification is that the author does not make any statement about the quality or value of the cited work. The citation simply presents information about the findings from the February and October surveys without expressing any personal opinion or judgment about the work "Angels in Plastic" or its authors. The focus is on the data and conclusions drawn from the surveys, maintaining a neutral stance on the cited material.


100%|██████████| 100/100 [04:14<00:00,  2.54s/it]

The label "opinionated" is appropriate for this citation because the author expresses their opinion by stating that among academics, the weakness of the UPC and its dominance by regional big men was an accepted fact. This statement reflects a judgment about the perception and understanding of the party within academic circles, indicating a subjective evaluation of the party's structure and dynamics. The author's assertion of this accepted fact introduces a level of interpretation and assessment, making the citation opinionated rather than neutral.


In [5]:
labels = df["Label"].tolist()

def calculate_accuracy_per_label(predictions, labels, label_value):
    # Create a boolean array indicating whether the label matches the specified value
    label_matches = [l == label_value for l in labels]
  
    # Extract predictions for instances where the label matches the specified value
    matched_predictions = [p for i, p in enumerate(predictions) if label_matches[i]]
        
    return sum(matched_predictions)/100 if label_value == 1 else (len(matched_predictions) - sum(matched_predictions))/100
   
f1 = f1_score(predictions, labels)
accuracy_label_0 = calculate_accuracy_per_label(predictions, labels, label_value=0)
accuracy_label_1 = calculate_accuracy_per_label(predictions, labels, label_value=1)

print("F1 score:", f1)
print("Accuracy for label 0:", accuracy_label_0)
print("Accuracy for label 1:", accuracy_label_1)

F1 score: 0.6644295302013422
Accuracy for label 0: 0.0
Accuracy for label 1: 0.99
